# Creating Links for Scraping

This script loops through the 5 different sections of the Stock X website. The intention is just to get the shoe name/colorway key that we would then use in the query to access each shoe's page. There were variables created for each of the sections of the website and the script was ran for each variable.

In [63]:
import os
import selenium
from selenium import webdriver
import time
from PIL import Image
import io
import requests
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
import datetime as dt
import pandas as pd
import csv
import collections
import io
import dropbox

In [37]:
# Install and initiate driver
driver = webdriver.Chrome(ChromeDriverManager().install())

# the start URL from the adidas brand page
adidas_url = 'https://stockx.com/adidas'

# open and maximize the browser window
driver.get(adidas_url)
driver.maximize_window()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 91.0.4472
[WDM] - Get LATEST driver version for 91.0.4472
[WDM] - Driver [/Users/gabbyvinco/.wdm/drivers/chromedriver/mac64/91.0.4472.19/chromedriver] found in cache


In [38]:
# initiate the action object
action = webdriver.ActionChains(driver)

# get rid of the pop up message upon opening the website
wait = WebDriverWait(driver, 10)
pop_up_confirm = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[6]/div[4]/div/section/footer/button')))
pop_up_confirm.click()

In [39]:
# create empty lists to add stuff to iterate through
all_xpaths = []
all_hrefs = []
shoe_link = []

In [40]:
# create xpath routes for all the links on the page
for tile in range (1,21):
    # no need to adjust tile path with the change of url
    tile_path = "/html/body/div[1]/div[1]/div[2]/div[2]/div[3]/div/div/div[2]/div[2]/div[2]/div[2]/div["+str(tile)+"]/div/a"
    all_xpaths.append(tile_path)
#print(all_xpaths)


In [41]:
# scrape all the href links for each tile on the page
def scrape_href():
    for path in all_xpaths:
        link = driver.find_element_by_xpath(path).get_attribute('href')
        all_hrefs.append(link)
    #print(all_hrefs)

In [42]:
# split to isolate the link needed for the request
def split_href():
    for href in all_hrefs:
        link = href.split("m/")
        shoe_link.append(link[1])
    #print("links added to list")

In [49]:
xpath_list = ['/html/body/div[1]/div[1]/div[1]/nav/div/div/ul/li[1]/ul/li[1]/ul/li[2]/a', #jordan tab xpath
             '/html/body/div[1]/div[1]/div[1]/nav/div/div/ul/li[1]/ul/li[1]/ul/li[3]/a',  #nike tab xpath
             '/html/body/div[1]/div[1]/div[1]/nav/div/div/ul/li[1]/ul/li[1]/ul/li[4]/a',  #other brands tab xpath
             '/html/body/div[1]/div[1]/div[1]/nav/div/div/ul/li[1]/ul/li[1]/ul/li[5]/a',  #luxury brands tab xpath
             '/html/body/div[1]/div[1]/div[1]/nav/div/div/ul/li[1]/ul/li[1]/ul/li[5]/a']  #luxury brands tab xpath 
             # we have the luxury brands twice so that they will be scraped otherwise the script will cut off before

In [50]:
# function to pass through the different links on the toolbar so then the scraping can 
# be continuous and not have to be re-ran 5 times
def switch_through_tabs(xpath):
    browse_tab = driver.find_element_by_xpath('/html/body/div[1]/div[1]/div[1]/nav/div/div/ul/li[1]/a')
    browse_tab.click()
    time.sleep(2)
    sneaker_tab = driver.find_element_by_xpath('/html/body/div[1]/div[1]/div[1]/nav/div/div/ul/li[1]/ul/li[1]/a')
    sneaker_tab.click()
    time.sleep(2)
    next_tab = driver.find_element_by_xpath(xpath)
    next_tab.click()
    time.sleep(3)
    

In [51]:
# test to see if it works before scraping everything
# for xpath in xpath_list:
#     switch_through_tabs(xpath)

In [52]:
# create a new list to hold all the links
cumulative_shoe_links = []

In [53]:
for xpath in xpath_list:
    # automate it to click through all the pages until we have a duplicate link
    page_count = 0
    tile_count = 0
    # make sure lists are empty before scraping
    all_hrefs = []
    shoe_link = []

    # range 1 to 51 because there are a maximum number of 50 pages to go through
    while page_count <= 30:
        page_count += 1 # add to count so you know how many iterations you're doing
        tile_count += 20 # add to tile count so you can check to see if the current page duplicates with the first page
        scrape_href() # get hrefs
        split_href() # split hrefs
        driver.refresh()
        # next button is also the same regardless of switch in url
        element = driver.find_element_by_xpath("/html/body/div[1]/div[1]/div[2]/div[2]/div[3]/div/div/div[2]/div[3]/ul/a[7]")
        driver.execute_script("arguments[0].click();", element) 
        # skip the first iteration becuase we know there aren't any duplicates on the first page
        if page_count == 1:
            print("working... scraping from this shoe tab.")
            continue
        # if not first iteration, then we need to check for duplicates
        elif page_count != 1:
            first_shoe = shoe_link[0]
            counter=collections.Counter(shoe_link)
            unique_shoes = list(counter.keys())
            if unique_shoes.count(first_shoe) > 1:
                print("duplicate found")
                break
            else:
#                 print("working")
                continue
    # add shoe_link for this tab to an overall list of all links
    cumulative_shoe_links.extend(shoe_link)
    print("proceed to next shoe tab")
    # switch to the next sneaker tab on the stock x            
    switch_through_tabs(xpath)
print("shoe key scrape complete")

working... scraping from this shoe tab.
proceed to next shoe tab
working... scraping from this shoe tab.
proceed to next shoe tab
working... scraping from this shoe tab.
proceed to next shoe tab
working... scraping from this shoe tab.
proceed to next shoe tab
working... scraping from this shoe tab.
proceed to next shoe tab
shoe key scrape complete


In [65]:
# exit from the driver window
driver.quit()

In [56]:
# check how many shoes we scraped (including duplicates)
len(cumulative_shoe_links)

49600

In [57]:
# removing duplicates from scraped shoes
counter = collections.Counter(cumulative_shoe_links)
all_unique_shoes = list(counter.keys())

In [58]:
# check how many shoes we scraped (only unique shoes)
len(all_unique_shoes)

2405

In [59]:
# make list to df
cumulative_list_df = pd.DataFrame(all_unique_shoes)

In [61]:
# save as csv to loop through later in requests
with open('cumulative_list.csv', 'w', newline='\n') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter = csvwriter.writerow(cumulative_list_df)

In [64]:
# save the csv to the project's dropbox folder

# running the notebook that contains the twitter api keys to pass the variables in
%run ./dropbox_key.ipynb
# connect to the group project dropbox
DBX = dropbox.Dropbox(dropbox_token)
# save df as csv file
data = cumulative_list_df.to_csv(index=False) # The index parameter is optional
# upload to the project's dropbox folder
with io.BytesIO(data.encode()) as stream:
    stream.seek(0)
    DBX.files_upload(stream.read(), "/cumulative_list", mode=dropbox.files.WriteMode.overwrite)